In [ ]:
import json
import pandas as pd
import psycopg2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

##2가지로 나눔 1: 전체 태그 반영 / 2: 일부 태그만 반영

###람다에 올릴 최종 함수

###취향 계산 함수는, 일부 태그만 반영하기 위해서 dataframe으로 변경하여 반환하도록 변경

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db
def calculate_taste(user_id,db):
  print("calculate_tast...")
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    #print('place : ',place,"\tnow : ",place_theme)

  avg_theme_score = theme_score / len(survey_result)
  user_taste = pd.DataFrame(avg_theme_score.reshape(1,-1))
  user_taste.columns = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
  print("calculate_tast DONE!!")
  return user_taste

In [ ]:
def load_place(db):
  cursor= db.cursor()
  
  load_place_sql= f'SELECT place_id,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster FROM places_analysis'
  cursor.execute(load_place_sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  df.columns = ['place_id','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster']
  df = df.sort_values(by='place_id',ascending=True)
  return df
def recommend_place(user_id,top):
  
  db = load_db()
  place = load_place(db)
  user_taste = calculate_taste(user_id,db)
  
  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
  

  #entire tag
  print(user_taste)
  entire_similarity = cosine_similarity(user_taste, place[tag])
  entire_similarity = entire_similarity[0]
      
  print('entire similarity : ',entire_similarity)
  
  result_index = entire_similarity.argsort()
  
  print("result_index : ",result_index)


  for i in range(-1,-4,-1):
    now_idx = result_index[i]
    sim = entire_similarity[now_idx]
    print("now place ",place.iloc[now_idx])
    print("similarity : ",sim,"df index : ",now_idx)
    

  top_N_result = result_index[-1:-(101):-1]
  print("top_N_similart_result",top_N_result[:10])
  
  result = place.iloc[top_N_result].sort_values(by='place_id',ascending=True)
  result = result.iloc[:top]
  print("result.iloc[:top]")
  print(result)

  result = random.sample(list(result['place_id']), 2) # if course에 있는거랑 곂치면 다시 뽑자!!!
  
  print("result **id** list",result)
  
  print("*"*50)

  #6개 태그만 적용하기
  #2개를 뽑아서 없앤다음, 적용하자.4개중 nature, exciting, culture, day
  six_tag = tag.copy()
  picking_list = ['nature', 'exciting', 'sea', 'day']
  randomly_chosen_two_tags = random.sample(picking_list,2)
  for tag in randomly_chosen_two_tags:
    six_tag.remove(tag) #뽑힌 2개는 지워주기

  user_tatse_six = user_tatse[six_tag]
  print("user taste six : ",user_tatse_six)
  six_tag_similarity = cosine_similarity(user_tatse_six, place[six_tag])
  six_tag_similarity = six_tag_similarity[0]
  
  print('six similarity : ',six_tag_similarity)
  
  six_result_index = six_tag_similarity.argsort()
  
  print("six result_index : ",six_result_index)

  for i in range(-1,-4,-1):
    now_idx = six_result_index[i]
    sim = six_tag_similarity[now_idx]
    print("now place ",place.iloc[now_idx])
    print("similarity : ",sim,"df index : ",now_idx)
    
  top_N_result_six = six_result_index[-1:-101:-1]
  print("top_N_result(only six tag)",top_N_result_six[:10])
  
  result_six = place.iloc[top_N_result_six].sort_values(by='place_id',ascending=True)
  result_six = result_six.iloc[:top]
  print("result _ six")
  print(result_six)
  
  result_six = random.sample(list(result_six['place_id']), 2) # if 위에서 뽑은거랑 곂치면 다시 뽑자!!!
  while set(result_six).intersection(result) != set():
    result_six = random.sample(list(result_six['place_id']), 2)

  
  final_result = result+result_six

  return final_result

In [ ]:
result = recommend_place(user_id=184,top=50)
print(result)


calculate_tast...
[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
calculate_tast DONE!!
   nature  outdoor  fatigue  sea  walking  exciting  day  culture
0     2.0      2.5      1.4  1.6      2.2       1.3  2.8      0.1
entire similarity :  [0.82417379 0.93669009 0.89572696 ... 0.42084073 0.55776359 0.84998234]
result_index :  [449 450 513 ... 607 516 440]
now place  place_id    441
nature        2
outdoor       2
fatigue       1
sea           1
walking       2
exciting      1
day           2
culture       0
cluster       8
Name: 453, dtype: int64
similarity :  0.990056184533482 df index :  440
now place  place_id    517
nature        2
outdoor       2
fatigue       1
sea           1
walking       2
exciting      1
day           2
culture       0
cluster       8
Name: 526, dtype: int64
similarity :  0.990056184533482 df index :  516
now place  place_id    608
nature        2
outdoor       2
fatigue       1
sea           1
walking       2
exciting      1
day           2
culture       0
cluster  

In [ ]:
def recommend_attraction_list(df,attraction_id,top):
  target_attraction_index = df[df['id']==attraction_id].index.values
  print(target_attraction_index)
  total_similarity = cosine_similarity(df.iloc[:,12:])
  print(total_similarity.shape)
  similarity = total_similarity[target_attraction_index]
  similarity = list(similarity[0])
    
  print('similarity : ',similarity)
  
  similarity_with_index = [(val,idx) for idx,val in enumerate(similarity)]
  similarity_with_index.sort(reverse = True) 
  print("similarity_with_index : ",similarity_with_index)

  result_index = [i[1] for i in similarity_with_index]
  print("result_index : ",result_index)

  #자기 자신은 제외 (1일것임)
  top_N_result = result_index[:top+1]
  print(top_N_result)
  

  result = df.iloc[top_N_result].sort_values(by='score',ascending=False)
  return result

여행지를 추천할때, 여행 코스는 애초에 종합적으로 고려가 되서 기존의 평균 방식으로 해도 심각한 문제는 없을 것임. 하지만 여행지를 추천할 때는, 성향을 평균내면 비슷한 여행지만 추천될것임. 이를 해결하기 위해서 
Nature를 제외한 나머지 요인만을 고려하여 score순으로 정렬한다음, 상위 20개만 뽑자. 거기서 random으로 1개 주는거임! 



8개를 다 고려해서 나온 결과 Top 10 -> 1개 random select

나머지 3개는 random으로 가즈아 시발!!


밤인것도 좀 넣어야함.

nature, exciting, sea, day



##실험실

In [ ]:
import random
tag_list = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
copy1 = tag_list.copy()
copy2 = tag_list.copy()
copy3 = tag_list.copy()
copy4 = tag_list.copy()
copy1.remove("nature")
copy2.remove("exciting")
copy3.remove("culture")
copy4.remove("day")

In [ ]:
#4개중 nature, exciting, culture, day
picking_list = ['nature', 'exciting', 'sea', 'day']
random_tests = random.sample(picking_list,2)
print(random_tests)
for i in random_tests:
  picking_list.remove(i)
print(picking_list)
# print(copy1)
# print(copy2)
# print(copy3)
# print(copy4)

['culture', 'nature']
['exciting', 'day']


###df 1line 불러와서 전체랑 유사도 검사 되는지 보자.

In [ ]:
db=load_db()
cursor=db.cursor()
load_place_sql= f'SELECT place_id,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster FROM places_analysis'
cursor.execute(load_place_sql)
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ['place_id','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster']
sample_place = df.sample()
sample_tag = sample_place[['nature','outdoor','fatigue','sea','walking','exciting','day','culture']]

In [ ]:
sample_tag

,nature,outdoor,fatigue,sea,walking,exciting,day,culture
330,0,0,2,1,1,3,3,0


In [ ]:
similarity_check = cosine_similarity(sample_tag,df[['nature','outdoor','fatigue','sea','walking','exciting','day','culture']])

In [ ]:
sorting_check = similarity_check.argsort()
print(sorting_check[-1])

[ 953  441   31 ...  330  116 1021]


In [ ]:
df.iloc[1021]

place_id    1016
nature         0
outdoor        0
fatigue        2
sea            1
walking        1
exciting       3
day            3
culture        0
cluster        4
Name: 1021, dtype: int64

In [ ]:
another_sample = df.sample()[['nature','outdoor','fatigue','sea','walking','exciting','day','culture']]
another_sample

,nature,outdoor,fatigue,sea,walking,exciting,day,culture
704,3,3,1,0,1,0,3,1


###취향 분석 함수 numpy array -> dataframe으로 변경

In [ ]:
def calculate_taste(user_id,db):
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    print('place : ',place,"\tnow : ",place_theme)

  avg_theme_score = theme_score / len(survey_result)
  user_taste = pd.DataFrame(avg_theme_score.reshape(1,-1))
  user_taste.columns = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
  return user_taste

In [ ]:
user_taste = calculate_taste(184,db)
user_taste

[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
place :  32 	now :  (2, 3, 1, 1, 3, 1, 3, 0)
place :  19 	now :  (2, 3, 1, 2, 3, 0, 3, 0)
place :  13 	now :  (2, 3, 1, 3, 3, 1, 3, 0)
place :  6 	now :  (3, 2, 1, 1, 2, 1, 3, 1)
place :  33 	now :  (0, 0, 1, 1, 0, 3, 2, 0)
place :  8 	now :  (2, 3, 2, 3, 2, 2, 3, 0)
place :  10 	now :  (2, 2, 1, 1, 3, 1, 2, 0)
place :  25 	now :  (2, 3, 2, 3, 3, 2, 3, 0)
place :  41 	now :  (2, 3, 1, 1, 1, 2, 3, 0)
place :  72 	now :  (3, 3, 3, 0, 2, 0, 3, 0)


,nature,outdoor,fatigue,sea,walking,exciting,day,culture
0,2.0,2.5,1.4,1.6,2.2,1.3,2.8,0.1


In [ ]:
user_taste = pd.DataFrame(user_taste.reshape(1,-1))
user_taste.columns = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
user_taste

,nature,outdoor,fatigue,sea,walking,exciting,day,culture
0,2.0,2.5,1.4,1.6,2.2,1.3,2.8,0.1


In [ ]:
user_taste[['nature','outdoor']]

,nature,outdoor
0,2.0,2.5


###np 유사도 비교

In [ ]:
np1 = np.array([2.0, 2.5, 1.4, 1.6, 2.2, 1.3, 2.8, 0.1]).reshape(1,-1)
np2 = np.array([2,2,1,1,2,1,2,0]).reshape(1,-1)
cosine_similarity(np1,np2)

array([[0.99005618]])

In [ ]:
np3 = np.array([2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 3.0, 0.0]).reshape(1,-1)
cosine_similarity(np1,np3)

array([[0.987367]])

In [ ]:
np3 = np.array([2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 3.0, 0.0]).reshape(1,-1)
cosine_similarity(np1,np3)

array([[0.987367]])

###교집합 none 체크

In [ ]:
set1 = [1,2,3]
set2 = [4,5,6]
set(set1).intersection(set2) == set()

True

##lambda에 올린 최종본

In [ ]:
import json
import pandas as pd
import psycopg2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random
import os

def load_db():
  endpoint = os.environ['END_POINT']
  dbname = os.environ['DB_NAME']
  user = os.environ['USER_NAME']
  password = os.environ['WONSEOK']
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db


def calculate_taste(user_id,db):
  print("calculate_tast...")
  cursor = db.cursor()
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*8)
  for place in survey_result:
    load_place_sql = f'SELECT nature,outdoor,fatigue,sea,walking,exciting,day,culture FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)
    #print('place : ',place,"\tnow : ",place_theme)

  avg_theme_score = theme_score / len(survey_result)
  user_taste = pd.DataFrame(avg_theme_score.reshape(1,-1))
  user_taste.columns = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
  print("calculate_tast DONE!!")
  return user_taste

def load_place(db):
  cursor= db.cursor()
  
  load_place_sql= f'SELECT place_id,nature,outdoor,fatigue,sea,walking,exciting,day,culture,cluster FROM places_analysis'
  cursor.execute(load_place_sql)
  result = cursor.fetchall()
  df = pd.DataFrame(result)
  df.columns = ['place_id','nature','outdoor','fatigue','sea','walking','exciting','day','culture','cluster']
  df = df.sort_values(by='place_id',ascending=True)
  return df
def recommend_place(user_id):
  
  top = 50
  db = load_db()
  place = load_place(db)
  user_taste = calculate_taste(user_id,db)
  
  tag = ['nature','outdoor','fatigue','sea','walking','exciting','day','culture']
  

  #entire tag
  print(user_taste)
  entire_similarity = cosine_similarity(user_taste, place[tag])
  entire_similarity = entire_similarity[0]
      
  print('entire similarity : ',entire_similarity)
  
  result_index = entire_similarity.argsort()
  
  print("result_index : ",result_index)


  for i in range(-1,-4,-1):
    now_idx = result_index[i]
    sim = entire_similarity[now_idx]
    print("now place ",place.iloc[now_idx])
    print("similarity : ",sim,"df index : ",now_idx)
    

  top_N_result = result_index[-1:-(101):-1]
  print("top_N_similart_result",top_N_result[:10])
  
  result = place.iloc[top_N_result].sort_values(by='place_id',ascending=True)
  result = result.iloc[:top]
  print("result.iloc[:top]")
  print(result)

  result = random.sample(list(result['place_id']), 2) # if course에 있는거랑 곂치면 다시 뽑자!!!
  
  print("result **id** list",result)
  
  print("*"*50)

  #6개 태그만 적용하기
  #2개를 뽑아서 없앤다음, 적용하자.4개중 nature, exciting, culture, day
  six_tag = tag.copy()
  picking_list = ['nature', 'exciting', 'sea', 'day']
  randomly_chosen_two_tags = random.sample(picking_list,2)
  for tag in randomly_chosen_two_tags:
    six_tag.remove(tag) #뽑힌 2개는 지워주기

  user_taste_six = user_taste[six_tag]
  print("user taste six : ",user_taste_six)
  six_tag_similarity = cosine_similarity(user_taste_six, place[six_tag])
  six_tag_similarity = six_tag_similarity[0]
  
  print('six similarity : ',six_tag_similarity)
  
  six_result_index = six_tag_similarity.argsort()
  
  print("six result_index : ",six_result_index)

  for i in range(-1,-4,-1):
    now_idx = six_result_index[i]
    sim = six_tag_similarity[now_idx]
    print("now place ",place.iloc[now_idx])
    print("similarity : ",sim,"df index : ",now_idx)
    
  top_N_result_six = six_result_index[-1:-101:-1]
  print("top_N_result(only six tag)",top_N_result_six[:10])
  
  result_six = place.iloc[top_N_result_six].sort_values(by='place_id',ascending=True)
  result_six = result_six.iloc[:top]
  print("result _ six")
  print(result_six)
  
  result_six = random.sample(list(result_six['place_id']), 2) # if 위에서 뽑은거랑 곂치면 다시 뽑자!!!
  while set(result_six).intersection(result) != set():
    result_six = random.sample(list(result_six['place_id']), 2)

  
  final_result = result+result_six

  return final_result


def lambda_handler(event, context):
  # TODO implement
  id = int(event['queryStringParameters']['id'])
    
  try:
    db = load_db()
  except:
    return {
            'statusCode': 500,
            "success": False,
            "message": "Database Error",
            "name":os.environ['DB_NAME'],
            "endpoint":os.environ['END_POINT'],
            "user_name":os.environ['USER_NAME'],
            "password":os.environ['WONSEOK'],
            
        }
        
  place_list = recommend_place(id)
  
  

  return {
      'statusCode': 200,
      'body': json.dumps({  
                            "placeList" : place_list,
                            })
  }